# Training yolov8 models on custom dataset

## Check GPU access

In [ ]:
!nvidia-smi

## Install and import necessary libraries

In [ ]:
!pip install ultralytics==8.0.134

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

## download the dataset from Roboflow [annotator tool]

In [ ]:
!mkdir datasets
%cd datasets

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="your-api-key-here")
project = rf.workspace("wce-fpcql").project("wce_clean_train_2.0")
dataset = project.version(1).download("yolov8")

## Training the model

In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data={dataset.location}/data.yaml epochs=50 imgsz=800 plots=True

## Validate the model

In [ ]:
!yolo task=detect mode=val model=/content/datasets/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

## Prediction / Inference with the model

In [ ]:
!yolo task=detect mode=predict model=/path/to/your/weights/best.pt conf=0.25 source=/path/to/your/test/images save=True

## IoU claculation

In [ ]:
import os

def yolo_to_coords(yolo_format):
    center_x, center_y, width, height = yolo_format
    x1 = (center_x - width / 2)
    y1 = (center_y - height / 2)
    x2 = (center_x + width / 2)
    y2 = (center_y + height / 2)
    return x1, y1, x2, y2

def calculate_iou(box1, box2):
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # Calculate the coordinates of the intersection rectangle
    x1_intersection = max(x1_1, x1_2)
    y1_intersection = max(y1_1, y1_2)
    x2_intersection = min(x2_1, x2_2)
    y2_intersection = min(y2_1, y2_2)
    
    # Calculate the area of the intersection rectangle
    intersection_area = max(0, x2_intersection - x1_intersection) * max(0, y2_intersection - y1_intersection)
    
    # Calculate the areas of the predicted and ground truth rectangles
    area_box1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area_box2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # Calculate the union area
    union_area = area_box1 + area_box2 - intersection_area
    
    # Calculate IoU
    iou = intersection_area / union_area
    
    return iou

def calculate_iou_for_folders(ground_truth_folder, predicted_labels_folder):
    iou_list = []
    
    # Get the list of filenames in the ground truth folder
    file_names = os.listdir(ground_truth_folder)
    
    for file_name in file_names:
        # Construct the full file paths for ground truth and predicted labels
        ground_truth_file_path = os.path.join(ground_truth_folder, file_name)
        predicted_file_path = os.path.join(predicted_labels_folder, file_name)
        
        # Read and process the files to calculate IoU
        with open(ground_truth_file_path, 'r') as ground_truth_file, open(predicted_file_path, 'r') as predicted_file:
            ground_truth_lines = ground_truth_file.readlines()
            predicted_lines = predicted_file.readlines()
        
        # Parse YOLO format lines into lists of bounding boxes
        ground_truth_bboxes = [list(map(float, line.strip().split()[1:])) for line in ground_truth_lines]
        predicted_bboxes = [list(map(float, line.strip().split()[1:])) for line in predicted_lines]
        
        # Convert YOLO format to (x1, y1, x2, y2) format for each bounding box
        ground_truth_bboxes = [yolo_to_coords(bbox) for bbox in ground_truth_bboxes]
        predicted_bboxes = [yolo_to_coords(bbox) for bbox in predicted_bboxes]
        
        # Calculate IoU for each pair of bounding boxes
        ious = []
        for predicted_bbox in predicted_bboxes:
            for ground_truth_bbox in ground_truth_bboxes:
                iou = calculate_iou(predicted_bbox, ground_truth_bbox)
                ious.append(iou)
        
        # Choose the maximum IoU as the final IoU for this case
        max_iou = max(ious)
        
        # Append the max IoU to the list
        iou_list.append(max_iou)
    
    # Calculate the mean IoU
    mean_iou = sum(iou_list) / len(iou_list)
    
    return iou_list, mean_iou

# Example usage:
ground_truth_folder = "/path/to/ground_truth_labels"
predicted_labels_folder = "/path/to/predicted_labels"
ious, mean_iou = calculate_iou_for_folders(ground_truth_folder, predicted_labels_folder)
# print("IoUs for all files:", ious)
print("Mean IoU:", mean_iou)
